In [1]:
from pprint import pprint

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import ry_prepare as p
import acquire_ry as a
import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import ry_wrangle as w

In [2]:
df = a.get_github(cached=True)

In [3]:
df.head()

,language,content
0,Batchfile,Environmental\nEnvironmental quickly build\n优秀...
1,R,seraphim \nseraphim is a R package for investi...
2,PHP,PHP dotenv\nA .env file parsing and loading li...
3,Ruby,\n\n\nMersea\nRequirements\n\nRuby MRI 2.5.x (...
4,C++,\nWifi and Bluetooth environmental sensors bui...


In [4]:
df = p.prep_data(df, 'content', extra_words=[], exclude_words=[])
df.head()

,language,content,clean,stemmed,lemmatized,words,doc_length
5,Java,\nWifi and Bluetooth environmental sensors bui...,wifi bluetooth environmental sensors built esp...,wifi bluetooth environment sensor built esp32 ...,wifi bluetooth environmental sensor built esp3...,"[wifi, bluetooth, environmental, sensor, built...",83
13,HTML,IndoPacific-Corals\nAnalysis of Indo-Pacific c...,indopacificcorals analysis indopacific coral l...,indopacificcor analysi indopacif coral life hi...,indopacificcorals analysis indopacific coral l...,"[indopacificcorals, analysis, indopacific, cor...",10
14,JavaScript,Grassberry High is an environmental controller...,grassberry high environmental controller home ...,grassberri high environment control home grow ...,grassberry high environmental controller home ...,"[grassberry, high, environmental, controller, ...",335
15,Java,Grassberry High is an environmental controller...,grassberry high environmental controller home ...,grassberri high environment control home grow ...,grassberry high environmental controller home ...,"[grassberry, high, environmental, controller, ...",335
16,JavaScript,Environmental Impact of COVID-19\nThis project...,environmental impact covid19 project examines ...,environment impact covid19 project examin pote...,environmental impact covid19 project examines ...,"[environmental, impact, covid19, project, exam...",249


In [5]:
train_exp, X_train, y_train, X_validate, y_validate, X_test, y_test = w.split(df,'language')
train_exp

,language,content,clean,stemmed,lemmatized,words,doc_length
151,HTML,environmental-app\n,environmentalapp,environmentalapp,environmentalapp,[environmentalapp],1
54,JavaScript,EnvironmentalMonitoring\narduino code\n,environmentalmonitoring arduino code,environmentalmonitor arduino code,environmentalmonitoring arduino code,"[environmentalmonitoring, arduino, code]",3
48,Java,Environmental Monitor\nData captured\nData is ...,environmental monitor data captured data publi...,environment monitor data captur data publish s...,environmental monitor data captured data publi...,"[environmental, monitor, data, captured, data,...",160
13,HTML,IndoPacific-Corals\nAnalysis of Indo-Pacific c...,indopacificcorals analysis indopacific coral l...,indopacificcor analysi indopacif coral life hi...,indopacificcorals analysis indopacific coral l...,"[indopacificcorals, analysis, indopacific, cor...",10
94,HTML,Greening a PostIndustrial City: Applying keyw...,greening postindustrial city applying keyword ...,green postindustri citi appli keyword extracto...,greening postindustrial city applying keyword ...,"[greening, postindustrial, city, applying, key...",935
...,...,...,...,...,...,...,...
167,JavaScript,Environmental-Logger\nLog environmental data s...,environmentallogger log environmental data tem...,environmentallogg log environment data tempera...,environmentallogger log environmental data tem...,"[environmentallogger, log, environmental, data...",19
105,Python,Corporate environmental performance prediction...,corporate environmental performance prediction...,corpor environment perform predict china empir...,corporate environmental performance prediction...,"[corporate, environmental, performance, predic...",102
180,HTML,environmental-science\nMy first website develo...,environmentalscience first website developed u...,environmentalsci first websit develop use html...,environmentalscience first website developed u...,"[environmentalscience, first, website, develop...",14
120,Java,sensor-puck\nThis is the repository for all co...,sensorpuck repository collateral related silic...,sensorpuck repositori collater relat silicon l...,sensorpuck repository collateral related silic...,"[sensorpuck, repository, collateral, related, ...",30


In [6]:
#df.language.value_counts().head(15)

In [7]:
#df.info()

In [8]:
#remove_lang_list = ['CoffeeScript','Other','Go','Stata','Lua','Kotlin','Objective-C','Shell','Vim script','Batchfile','Swift','PowerShell','Vue', 'Mathematica','TypeScript']

In [9]:
#df = df[~df.language.isin(remove_lang_list)]

In [10]:
print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

(61, 6)
(21, 6)
(21, 6)


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Create CountVectorizer, which create bag-of-words model.
# stop_words : Specify language to remove stopwords. 
# min_df: ignore terms that have a document frequency strictly 
# lower than the given threshold. This value is also called cut-off in the literature. 
# If float, the parameter represents a proportion of documents, integer absolute counts. 
# ngram_range: the lower and upper boundary of the range of n-values for 
# different word n-grams or char n-grams to be extracted. 

vectorizer = CountVectorizer(stop_words='english', 
                             min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

# Learn vocabulary in sentences. 
vectorizer.fit(X_train.clean)

# Get dictionary. 
vectorizer.get_feature_names()

['data', 'environmental']

In [12]:
# Transform each sentences in vector space.
bow = vectorizer.transform(X_train.clean)

In [13]:
# this is just to see the array of 0's and 1's
bow_array = bow.toarray()
bow_array[3]

array([0, 0])

In [14]:
# Show sentences and vector space representation.
# purely to visualize what's happening.
for i, v in zip(X_train.clean, bow_array):
    print(i)
    print(v)

environmentalapp
[0 0]
environmentalmonitoring arduino code
[0 0]
environmental monitor data captured data published serial bus per second comma delimited string example 267035736101359126421 temperature celsius humidity percent light analog value 0 1023 pressure pa altitude accurate crc16 checksum data fields sensors dht11 temperature humidity bmp085 temperature pressure ldr light dependencies pip3 install pahomqtt pyserial crcmod config serial 9600 ldr a0 dht11 4 bmp085 0x77 publishing mqtt sertomqttpy takes data published arduino forwards mqtt broker takes name serial port name room parameters python3 sertomqttpy 1921681111 devttyacm0 livingroom alternative python q versionsertomqttq q sertomqttq q 1921681111 devttyacm0 livingroom subscribe command line confirm data publishing broker mosquittosub h 1921681111 hassio add sensors home assistant add configurationyaml sensor platform mqtt name living room temperature statetopic hassiolivingroomtemperature qos 0 unitofmeasurement oc sens

In [15]:
X_bow = bow

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', min_df=20, 
                             ngram_range=(1,2), 
                             binary=True)

tfidf_sparse_matrix = tfidf.fit_transform(X_train.clean)
tfidf_sparse_matrix

<61x2 sparse matrix of type '<class 'numpy.float64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [17]:
pd.DataFrame(tfidf_sparse_matrix.todense(), columns=tfidf.get_feature_names())

,data,environmental
0,0.000000,0.000000
1,0.000000,0.000000
2,0.743509,0.668726
3,0.000000,0.000000
4,0.000000,1.000000
...,...,...
56,0.743509,0.668726
57,0.000000,1.000000
58,0.000000,0.000000
59,0.000000,0.000000


In [18]:
# Get vocabularies.
tfidf.vocabulary_

{'environmental': 1, 'data': 0}

In [19]:
# Transform to document-term matrix
vector_spaces = tfidf.transform(X_train.clean)
vector_spaces.toarray()

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.74350931, 0.66872558],
       [0.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.74350931, 0.66872558],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.74350931, 0.66872558],
       [0.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 1.        ],
       [0.74350931, 0.66872558],
       [0.74350931, 0.66872558],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 0.        ],
       [1.        , 0.        ],
       [0.

In [20]:
# Show sentences and vector space representation.
# 
# (A, B) C
# A : Document Index
# B : Specific word-vector index
# C : TF-IDF score
for i, v in zip(X_train.clean, vector_spaces):
    print(i)
    print(v)

environmentalapp

environmentalmonitoring arduino code

environmental monitor data captured data published serial bus per second comma delimited string example 267035736101359126421 temperature celsius humidity percent light analog value 0 1023 pressure pa altitude accurate crc16 checksum data fields sensors dht11 temperature humidity bmp085 temperature pressure ldr light dependencies pip3 install pahomqtt pyserial crcmod config serial 9600 ldr a0 dht11 4 bmp085 0x77 publishing mqtt sertomqttpy takes data published arduino forwards mqtt broker takes name serial port name room parameters python3 sertomqttpy 1921681111 devttyacm0 livingroom alternative python q versionsertomqttq q sertomqttq q 1921681111 devttyacm0 livingroom subscribe command line confirm data publishing broker mosquittosub h 1921681111 hassio add sensors home assistant add configurationyaml sensor platform mqtt name living room temperature statetopic hassiolivingroomtemperature qos 0 unitofmeasurement oc sensor 2 platf

In [21]:
X_tfidf = tfidf_sparse_matrix

In [22]:
X_bow
X_tfidf

<61x2 sparse matrix of type '<class 'numpy.float64'>'
	with 47 stored elements in Compressed Sparse Row format>

In [34]:
y_train.value_counts()

language  
JavaScript    17
Python        16
Java          15
HTML          13
dtype: int64

In [24]:
from sklearn.linear_model import LogisticRegression

lm = LogisticRegression().fit(X_bow, y_train)

X_train['predicted'] = lm.predict(X_bow)
# test['predicted'] = lm.predict(X_test)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [61]:
baseline = X_train.copy()
baseline['b_predictions'] = baseline.predicted == 'JavaScript'
baseline

AttributeError: 'DataFrame' object has no attribute 'b_predictions'

In [26]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(train_exp.language, X_train.predicted)

array([[9, 4, 0, 0],
       [8, 3, 4, 0],
       [6, 2, 9, 0],
       [4, 4, 8, 0]])

In [27]:
confusion_matrix(y_train.language, X_train.predicted)

array([[9, 4, 0, 0],
       [8, 3, 4, 0],
       [6, 2, 9, 0],
       [4, 4, 8, 0]])

In [31]:
pd.crosstab(X_train.predicted, y_train.language)

language,HTML,Java,JavaScript,Python
predicted,,,,
HTML,9,8,6,4
Java,4,3,2,4
JavaScript,0,4,9,8


In [32]:
print(classification_report(y_train.language, X_train.predicted))

              precision    recall  f1-score   support

        HTML       0.33      0.69      0.45        13
        Java       0.23      0.20      0.21        15
  JavaScript       0.43      0.53      0.47        17
      Python       0.00      0.00      0.00        16

    accuracy                           0.34        61
   macro avg       0.25      0.36      0.28        61
weighted avg       0.25      0.34      0.28        61



/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
